In [ ]:
import os

from matplotlib import ticker
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from mpl_toolkits.mplot3d.art3d import pathpatch_2d_to_3d
import numpy as np
import pandas as pd

from dosipy.utils.viz import (fig_config, set_axes_equal, set_colorblind,
                              save_fig)

from utils import load_processed_data

In [ ]:
set_colorblind()
%config InlineBackend.figure_format = 'retina'

In [ ]:
# CST data

data = [['DipoleVertical', 5, [-0.009196137125833444, -0.008943818176342983], 2.9700],
        ['DipoleVertical', 10, [-0.01052339327664423, -0.010288400579897647], 2.1296],
        ['DipoleVertical', 15, [-0.01052339327664423, -0.010288400579897647], 1.4455],
        ['ArrayVertical', 5, [-0.010974217437555025, -0.0110377357372408], 5.6228],
        ['ArrayVertical', 10, [-0.008974217437555027, -0.009037735737240808], 5.9208],
        ['ArrayVertical', 15, [-0.008974686708799248, -0.009037296281864424], 5.2251]]

ds = pd.DataFrame(data, columns=['antenna', 'distance', 'origin', 'APD_CST'])
ds

In [ ]:
# visuals - apd distributions

for i in range(ds.shape[0]):
    antenna = ds.loc[i, 'antenna']
    distance = ds.loc[i, 'distance']
    origin = ds.loc[i, 'origin']
    edge = 0.02

    df = load_processed_data(antenna, distance)
    xyz = df[['x', 'y', 'z']].values
    PD = df['PD'].values

    skip = 1
    scaler = 2
    fig_config(latex=True, text_size=24, scaler=scaler)
    fig = plt.figure()
    ax = plt.axes(projection ='3d')

    # point cloud and apd spatial distribution
    cs = ax.scatter(-xyz[::skip, 0], -xyz[::skip, 1], xyz[::skip, 2],
                    c=PD[::skip], cmap='rocket', s=0.75)

    # averaging surface
    rect_pos = xyz[:, 0].max()
    patch_rect = Rectangle(origin, edge, edge,
                           ec='w', fc='None', lw=3)
    ax.add_patch(patch_rect)
    pathpatch_2d_to_3d(patch_rect, z=rect_pos, zdir='x')

    # colorbar
    cbar = fig.colorbar(cs, shrink=0.65, pad=-0.05)
    tick_locator = ticker.MaxNLocator(nbins=4)
    cbar.locator = tick_locator
    cbar.update_ticks()
    if i in [2, 5]:
        cbar.ax.set_ylabel('power density [W/m$^{2}$]')
        cbar.ax.yaxis.labelpad = 20

    # additional figure configuration
    ax.set(xlabel='',
           ylabel='',
           zlabel='',
           xticks=[],
           yticks=[-0.02, 0, 0.02],
           zticks=[-0.02, 0, 0.02],
           yticklabels=[-2, 0, 2],
           zticklabels=[-2, 0, 2])
    ax.xaxis.pane.fill = False
    ax.yaxis.pane.fill = False
    ax.zaxis.pane.fill = False
    ax.view_init(1, 0)
    if i in [3, 4, 5]:
        ax.set_ylabel('$y$ [cm]')
        ax.yaxis.labelpad = 20
    if i in [0, 3]:
        ax.zaxis.set_rotate_label(False)
        ax.set_zlabel('$z$ [cm]', rotation=90)
        ax.zaxis.labelpad = 20

    # set axes to be equal in size
    ax.set_box_aspect([1, 1, 1])
    ax = set_axes_equal(ax)
    fig.tight_layout();

    # save figure
    fname = os.path.join('figures', f'{antenna}_d{distance}mm')
    save_fig(fig, fname=fname, formats=['png'])